In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:11:08 of 0:11:08 remaining]
Chain 1:  10% [0:00:40 of 0:00:44 remaining]
Chain 1:  20% [0:00:49 of 0:01:01 remaining]
Chain 1:  30% [0:00:47 of 0:01:07 remaining]
Chain 1:  40% [0:00:41 of 0:01:08 remaining]
Chain 1:  50% [0:00:35 of 0:01:10 remaining]
Chain 1:  60% [0:00:29 of 0:01:12 remaining]
Chain 1:  70% [0:00:22 of 0:01:14 remaining]
Chain 1:  80% [0:00:15 of 0:01:15 remaining]
Chain 1:  90% [0:00:08 of 0:01:16 remaining]
Chain 1: 100% [0:00:00 of 0:01:17 remaining]

Chain 2:   0% [0:00:06 of 0:00:06 remaining]
Chain 2:  10% [0:00:58 of 0:01:04 remaining]
Chain 2:  20% [0:01:01 of 0:01:17 remaining]
Chain 2:  30% [0:00:53 of 0:01:16 remaining]
Chain 2:  40% [0:00:46 of 0:01:16 remaining]
Chain 2:  50% [0:00:37 of 0:01:14 remaining]
Chain 2:  60% [0:00:29 of 0:01:13 remaining]
Chain 2:  70% [0:00:22 of 0:01:13 remaining]
Chain 2:  80% [0:00:15 of 0:01:13 remaining]
Chain 2:  90% [0:00:07 of 0:01:12 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[26.5255 114.908 0.885622; 26.7598 115.962 0.856221; … ; 26.5527 112.791 0.92793; 25.1085 112.714 0.925671]

[32.1223 115.012 0.87638; 32.1687 113.69 0.904673; … ; 25.9912 116.215 0.858033; 28.9578 116.215 0.858033]

[24.6686 115.34 0.878248; 24.7454 115.269 0.872129; … ; 25.398 115.943 0.856483; 23.9178 115.943 0.856483]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD        Naive SE       MCSE        ESS   
     s2  26.1816100 2.00624984 0.01220964772 0.0212801009 8888.3811
beta[1] 113.9434298 1.94977710 0.01186596558 0.0387190128 2535.8373
beta[2]   0.9036523 0.04294051 0.00026132762 0.0008485271 2560.9608

Quantiles:
            2.5%         25.0%       50.0%      75.0%        97.5%   
     s2  22.56733705  24.79554725  26.069384  27.4499623  30.47020517
beta[1] 110.09170490 112.62812666 113.952305 115.2498748 117.76741684
beta[2]   0.81981035   0.87482183   0.903560   0.9326979   0.98782055



End of `clip_38.0m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*